# 0) IMPORTS

In [2]:
# Setup autoreload - automatically reload modules when they change
%load_ext autoreload
%autoreload 2

In [3]:
# Verify environment setup
import os
import sys

print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(f"\nEnvironment variables loaded:")
print(f"OPENAI_API_KEY: {'✓ Set' if os.getenv('OPENAI_API_KEY') else '✗ Not set'}")
print(f"POLYGON_API_KEY: {'✓ Set' if os.getenv('POLYGON_API_KEY') else '✗ Not set'}")


Python version: 3.12.7 (main, Oct  1 2024, 02:05:46) [Clang 16.0.0 (clang-1600.0.26.3)]
Python executable: /Users/realmistic/Documents/stocks-scoring-agent/.venv/bin/python

Environment variables loaded:
OPENAI_API_KEY: ✓ Set
POLYGON_API_KEY: ✓ Set


# 1) TEST: Checking news retrieval for the recent 5k records 
* The datasource is Massive.com (previously known as Polygon.io)
* 2 tools defined: 
  * search_news_by_ticker (not just YahooFinance source)
  * search_news_by_query (it can find a wider set of news to expand the topic)


In [4]:
from datetime import datetime, timezone
now = datetime.now(timezone.utc).strftime("%Y-%m-%d")
print (f'Time now = {now}')

# The exact time of running the Colab (in UTC, timezone-aware)
now_right_format = datetime.now(timezone.utc).isoformat(timespec='milliseconds')
print(f'Time now in the right format (for API) = {now_right_format}')

Time now = 2025-12-18
Time now in the right format (for API) = 2025-12-18T22:55:14.183+00:00


In [5]:
import requests
import pandas as pd
import os

# https://polygon.io/docs/stocks/get_v2_reference_news
# https://polygon.io/blog/api-pagination-patterns/
# API CALL : # https://api.polygon.io/v2/reference/news?order=desc&limit=1000&sort=published_utc&apiKey=<your key> or POLYGON_API_KEY
      # need to get 200 OK status

# check the API KEY is imported correctly from the .envrc file
print(len(f'Length of the POLYGON_API_KEY = {os.getenv("POLYGON_API_KEY")}'))

POLYGON_API_KEY = os.getenv('POLYGON_API_KEY')

# retrieve max 1000 news via one API call
#  Polygon.io is now Massive.com
def get_one_chunk_of_news_polygon_io(api_key = POLYGON_API_KEY, news_limit=1000, max_date = now):
#   url = f"https://api.polygon.io/v2/reference/news?order=desc&limit={news_limit}&sort=published_utc&published_utc.lt={max_date}&apiKey={api_key}"
  url = f"https://api.massive.com/v2/reference/news?order=desc&limit={news_limit}&sort=published_utc&published_utc.lt={max_date}&apiKey={api_key}"


  # https://www.nylas.com/blog/use-python-requests-module-rest-apis/ - Python for rest APIs
  # try/catch for HTTP requests: https://stackoverflow.com/questions/16511337/correct-way-to-try-except-using-python-requests-module
  try:
      url_sanitized = url.split("&apiKey=")[0] + "&apiKey=<your key>"
      print(f'trying the API call : {url_sanitized}')
      r = requests.get(url, timeout=3)
      r.raise_for_status()
  except requests.exceptions.HTTPError as errh:
      print ("Http Error:",errh)
  except requests.exceptions.ConnectionError as errc:
      print ("Error Connecting:",errc)
  except requests.exceptions.Timeout as errt:
      print ("Timeout Error:",errt)
  except requests.exceptions.RequestException as err:
      print ("OOps: Something Else",err)

  data = r.json()


  # Check if 'results' key exists in the response
  if 'results' in data:
      # If it exists, proceed with normalization
      df_nested_list = pd.json_normalize(data, record_path=['results'])
  else:
      # If not, print the response keys for debugging and create an empty DataFrame
      print(f"The 'results' key was not found in the response. Available keys are: {data.keys()}")
      df_nested_list = pd.DataFrame() # or handle it differently based on the new structure

  return df_nested_list

def get_all_news(api_calls_left = 5, api_key = POLYGON_API_KEY, news_limit=1000, max_date = now):
  all_news = None
  for i in range(api_calls_left):
    cur = get_one_chunk_of_news_polygon_io(api_key = api_key, news_limit = news_limit, max_date = max_date)
    if all_news is None:
      all_news = cur
    else:
      all_news = pd.concat([all_news,cur], ignore_index=True, axis=0) #stacking dataframes

    max_date = cur.published_utc.min() #update max_date of the news
  return all_news

64


In [6]:
# test getting all news
# 5 calls per minute limit for a free account - all recent news (5000)
all_news = get_all_news()

trying the API call : https://api.massive.com/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-12-18&apiKey=<your key>
trying the API call : https://api.massive.com/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-12-12T14:01:19Z&apiKey=<your key>
trying the API call : https://api.massive.com/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-12-07T17:01:00Z&apiKey=<your key>
trying the API call : https://api.massive.com/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-12-02T13:00:00Z&apiKey=<your key>
trying the API call : https://api.massive.com/v2/reference/news?order=desc&limit=1000&sort=published_utc&published_utc.lt=2025-11-25T15:20:00Z&apiKey=<your key>


In [7]:
all_news.head()

,id,title,author,published_utc,article_url,tickers,image_url,description,keywords,insights,publisher.name,publisher.homepage_url,publisher.logo_url,publisher.favicon_url,amp_url
0,0aa943d27178ad2e704f05d597d0a2788ce8f11b13ca80...,Kalaris Therapeutics Announces Oversubscribed ...,Corey Davis,2025-12-17T22:59:00Z,https://www.globenewswire.com/news-release/202...,"[KLRS, MS, MSpA, MSpE, MSpF, MSpI, MSpK, MSpL,...",https://ml.globenewswire.com/Resource/Download...,Kalaris Therapeutics secured a $50 million pri...,"[private placement, retinal diseases, TH103, n...","[{'ticker': 'KLRS', 'sentiment': 'positive', '...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.massive.com/public/assets/news/logo...,https://s3.massive.com/public/assets/news/favi...,NaN
1,edbe93ab88ae9e3de27d21ffedeb7a7f03b58015ff3648...,Prediction: This AI Stock Could Be the Next $4...,Will Healy,2025-12-17T22:30:00Z,https://www.fool.com/investing/2025/12/17/pred...,"[GOOG, GOOGL, MSFT, AAPL, NVDA]",https://g.foolcdn.com/image/?url=https%3A%2F%2...,Alphabet is predicted to become the next $4 tr...,"[AI, market cap, technology, investment, Gemin...","[{'ticker': 'GOOG', 'sentiment': 'positive', '...",The Motley Fool,https://www.fool.com/,https://s3.massive.com/public/assets/news/logo...,https://s3.massive.com/public/assets/news/favi...,NaN
2,d8efcc8ed6d49bee87fe24782ea5975131c27f2d6d8244...,ArrowMark Financial Corp. Releases Month End E...,Globe Newswire,2025-12-17T22:30:00Z,https://www.benzinga.com/pressreleases/25/12/g...,[BANX],https://www.benzinga.com/next-assets/images/be...,ArrowMark Financial Corp. announced its estima...,"[Net Asset Value, financial institutions, clos...","[{'ticker': 'BANX', 'sentiment': 'neutral', 's...",Benzinga,https://www.benzinga.com/,https://s3.massive.com/public/assets/news/logo...,https://s3.massive.com/public/assets/news/favi...,NaN
3,b16bcb0f66f99caf1b48e9ebe18ec12e6229eafd2ace50...,Nukkleus Inc. Receives Shareholder Approval to...,Globe Newswire,2025-12-17T22:30:00Z,https://www.benzinga.com/pressreleases/25/12/g...,"[NUKK, NUKKW]",https://www.benzinga.com/next-assets/images/be...,Nukkleus Inc. has received shareholder approva...,"[acquisition, defense technology, UAV, UAS, ae...","[{'ticker': 'NUKK', 'sentiment': 'positive', '...",Benzinga,https://www.benzinga.com/,https://s3.massive.com/public/assets/news/logo...,https://s3.massive.com/public/assets/news/favi...,NaN
4,655c04eb3dc48895eb95820e774c7de020100ec35d3094...,Nukkleus Inc. Receives Shareholder Approval to...,Menny Shalom,2025-12-17T22:30:00Z,https://www.globenewswire.com/news-release/202...,"[NUKK, NUKKW]",https://ml.globenewswire.com/Resource/Download...,Nukkleus Inc. has approved the acquisition of ...,"[aerospace, defense, UAV, UAS, acquisition, au...","[{'ticker': 'NUKK', 'sentiment': 'positive', '...",GlobeNewswire Inc.,https://www.globenewswire.com,https://s3.massive.com/public/assets/news/logo...,https://s3.massive.com/public/assets/news/favi...,NaN


In [8]:
all_news_json = all_news.to_dict(orient='records')

In [9]:
len(all_news_json)

5000

In [10]:
# Key features:

#   1. No chunking needed - Each news article is already a discrete document
#   2. Text search - Searches across title, description, keywords, author
#   3. Exact ticker matching - Function to find articles with specific ticker in tickers field
#   4. Multi-ticker support - Search for multiple tickers at once
#   5. Date filtering - Can filter news by publication date
#   6. Boosting - Title matches get higher priority than description

#   Note on chunking:
#   Unlike the podcast data (long markdown documents), news articles are already atomic units. Each article is a complete piece of information, so no chunking is
#   needed. The tickers field already provides precise ticker associations.

In [11]:

# Import minsearch
from minsearch import Index

# Step 1: Convert DataFrame to list of dicts (if not already done)
news_documents = all_news.to_dict(orient='records')

print(f"Total news articles: {len(news_documents)}")
print(f"Fields per article: {len(news_documents[0])}")
print(f"Sample article keys: {list(news_documents[0].keys())}")

Total news articles: 5000
Fields per article: 15
Sample article keys: ['id', 'title', 'author', 'published_utc', 'article_url', 'tickers', 'image_url', 'description', 'keywords', 'insights', 'publisher.name', 'publisher.homepage_url', 'publisher.logo_url', 'publisher.favicon_url', 'amp_url']


In [12]:
from tqdm import tqdm
  # Step 2: Preprocess documents - convert list fields to strings
print("\nPreprocessing documents...")
for doc in tqdm(news_documents, desc="Converting fields"):
    # Convert list fields to comma-separated strings
    if isinstance(doc.get('tickers'), list):
        doc['tickers'] = ', '.join(doc['tickers'])

    if isinstance(doc.get('keywords'), list):
        doc['keywords'] = ', '.join(doc['keywords'])

    # Ensure text fields are strings
    for field in ['title', 'description', 'author']:
        if doc.get(field) is None:
            doc[field] = ''
        elif not isinstance(doc.get(field), str):
            doc[field] = str(doc[field])

print("✓ Preprocessing complete")


Preprocessing documents...


Converting fields: 100%|██████████| 5000/5000 [00:00<00:00, 821156.66it/s]

✓ Preprocessing complete


In [13]:

  # Step 3: Create Index with relevant text fields
news_index = Index(
    text_fields=["title", "description", "keywords", "author", "tickers"],
    keyword_fields=["published_utc", "publisher.name"]
)

# Step 4: Fit the index with preprocessed documents
print("\nBuilding search index...")
news_index.fit(news_documents)
print(f"✓ Index created with {len(news_documents)} articles")


Building search index...
✓ Index created with 5000 articles


In [14]:
# Step 5: Search function to retrieve news for a ticker
def search_news_by_ticker(ticker, num_results=30):
    """
      Search for news articles related to a specific ticker.
      
      Args:
          ticker: Stock ticker symbol (e.g., 'TSLA', 'AAPL')
          num_results: Number of results to return
      
      Returns:
          List of news articles matching the ticker
    """
    results = news_index.search(
        query=ticker,
        num_results=num_results,
        boost_dict={
              'tickers': 5.0,      # Highest boost for ticker field
              'title': 3.0,         # High boost for title
              'description': 1.5,   # Medium boost for description
              'keywords': 1.0       # Standard boost for keywords
          }
      )
    return results

In [15]:
# Step 6: Search function to retrieve news for a search query
def search_news_by_query(query, num_results=30):
    """
      Search for news articles related to a specific query.
      
      Args:
          query: Search query (e.g., 'Tesla', 'AI robotics')
          num_results: Number of results to return
      
      Returns:
          List of news articles matching the ticker
    """
    results = news_index.search(
        query=query,
        num_results=num_results,
        boost_dict={
              'tickers': 1.0,      # boost for ticker field
              'title': 3.0,         # High boost for title
              'description': 5,   # Highest boost for description
              'keywords': 5       # Highest boost for keywords
          }
      )
    return results

In [16]:

# Step 7: Test - Get news for TSLA
print("\n" + "="*70)
print("Testing search for TSLA...")
print("="*70)

tsla_news = search_news_by_ticker("TSLA", num_results=5)

print(f"\nFound {len(tsla_news)} news articles for TSLA:\n")
for i, article in enumerate(tsla_news, 1):
    print(f"{i}. {article['title']}")
    print(f"   Published: {article['published_utc']}")
    print(f"   Tickers: {article['tickers']}")
    print(f"   Description: {article['description'][:100]}...")
    print()


Testing search for TSLA...

Found 5 news articles for TSLA:

1. Former Hedge Fund Manager James Altucher Says Musk’s Starlink Is Approaching a Major Turning Point
   Published: 2025-11-21T21:44:00Z
   Tickers: TSLA
   Description: Technology analyst James Altucher suggests Starlink is on the verge of a major public announcement, ...

2. What's Going On With The Uptick In Tesla Stock?
   Published: 2025-12-12T19:33:35Z
   Tickers: TSLA
   Description: Tesla stock traded higher after board member Kimbal Musk sold 56,820 shares worth $25.6 million. The...

3. Why Is Wall Street So Bearish on Tesla? There's 1 Key Reason.
   Published: 2025-12-05T05:16:00Z
   Tickers: TSLA
   Description: Tesla faces Wall Street skepticism due to its high valuation, declining operating margins, and uncer...

4. If You'd Invested $3,500 in Tesla 12 Years Ago, Here's How Much You'd Have Today
   Published: 2025-12-09T19:05:00Z
   Tickers: TSLA
   Description: An analysis of Tesla's stock performance reveals 

In [ ]:
# Step 8: search news by query
print("\n" + "="*70)
print("Testing search by query...")
print("="*70)

query = "Tesla competitors analysis EV margins autonomy AI robotics China market share outlook"
tsla_competitors_news = search_news_by_query(query, num_results=25)

print(f"\nFound {len(tsla_competitors_news)} news articles for TSLA competitors:\n")
for i, article in enumerate(tsla_competitors_news, 1):
    print(f"{i}. {article['title']}")
    print(f"   Published: {article['published_utc']}")
    print(f"   Tickers: {article['tickers']}")
    print(f"   Description: {article['description'][:100]}...")
    print()


Testing search by query...

Found 25 news articles for TSLA competitors:

1. Tesla Stock Stuck in Consolidation as Market Awaits Direction
   Published: 2025-12-03T06:38:00Z
   Tickers: TSLA
   Description: Tesla stock remains in a consolidation phase, trading near $429, with mixed signals from EV demand c...

2. Tesla Hits Ceiling In China? 2025 Sales Slump Could Mark A First For EV Giant
   Published: 2025-12-10T23:06:54Z
   Tickers: TSLA, BYDDY
   Description: Tesla faces potential first annual sales decline in China for 2025, struggling with consumer demand ...

3. 3 Signs Tesla Is Starting December on the Front Foot
   Published: 2025-12-05T03:42:00Z
   Tickers: TSLA, MFG, SF, SFB, SFpB, SFpC, SFpD
   Description: Tesla shows signs of recovery in December with technical strength, positive analyst sentiment, and i...

4. The Hits Keep on Coming for Tesla Investors
   Published: 2025-11-22T23:14:00Z
   Tickers: TSLA, F, FpB, FpC, FpD, GM
   Description: Tesla experienced significan

# 2) TEST: Checking different API endpoints (Yahoo Finance)

In [18]:
# https://github.com/ranaroussi/yfinance
# https://algotrading101.com/learn/yfinance-guide/

import yfinance as yf
ticker="TSLA"
ticker_obj = yf.Ticker(ticker)

In [19]:
# daily stats for a stock: Open, High, Low, Close, Volume
historical_prices = ticker_obj.history(period="2y", interval="1d")

# show only date part in the index
historical_prices.index = historical_prices.index.date

historical_prices.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2023-12-19,253.479996,258.339996,253.009995,257.220001,106737400,0.0,0.0
2023-12-20,256.410004,259.839996,247.000000,247.139999,125097000,0.0,0.0
2023-12-21,251.899994,254.800003,248.550003,254.500000,109594200,0.0,0.0
2023-12-22,256.760010,258.220001,251.369995,252.539993,93370100,0.0,0.0
2023-12-26,254.490005,257.970001,252.910004,256.609985,86892400,0.0,0.0


In [20]:
# Analyst price targets
ticker_obj.get_analyst_price_targets()

{'current': 483.37,
 'high': 600.0,
 'low': 120.0,
 'mean': 395.726,
 'median': 422.5}

In [21]:
# Dividends or Stock splits history
ticker_obj.actions

,Dividends,Stock Splits
Date,,
2020-08-31 00:00:00-04:00,0.0,5.0
2022-08-25 00:00:00-04:00,0.0,3.0


In [22]:
# Next earnings date + targets on EPS (Earnings)/revenue
ticker_obj.calendar

{'Earnings Date': [datetime.date(2026, 1, 28)],
 'Earnings High': 0.73,
 'Earnings Low': 0.14,
 'Earnings Average': 0.45236,
 'Revenue High': 28913000000,
 'Revenue Low': 21164000000,
 'Revenue Average': 25172640520}

In [23]:
# FIN STATEMENTS -- too much of info, not needed for now
# ticker_obj.get_balancesheet()
# ticker_obj.get_cash_flow()
# ticker_obj.get_income_stmt()
# same as income stmt: ticker_obj.get_financials()

In [24]:
# EPS estimates vs. numberOfAnalysts
ticker_obj.get_earnings_estimate()

,avg,low,high,yearAgoEps,numberOfAnalysts,growth
period,,,,,,
0q,0.45236,0.14000,0.73,0.73000,25,-0.3803
+1q,0.43977,0.27725,0.57,0.27000,12,0.6288
0y,1.65034,1.15999,2.03,2.42000,35,-0.3180
+1y,2.24805,1.33000,3.80,1.65034,34,0.3622


In [25]:
# How analysts have revised their EPS estimates over time?
ticker_obj.get_eps_revisions()

,upLast7days,upLast30days,downLast30days,downLast7Days
period,,,,
0q,1,4,2,1
+1q,2,2,5,5
0y,1,2,3,1
+1y,0,3,1,1


In [26]:
# This section retrieves consensus analyst growth expectations from Yahoo Finance for the selected ticker. 
# The values represent estimated earnings growth and are expressed in decimal form, 
# where positive values indicate expected growth and negative values indicate contraction. 
# Estimates are provided across multiple time horizons: 0q refers to the current fiscal quarter, 
# +1q to the next fiscal quarter, 0y to the current fiscal year, and +1y to the next fiscal year.
#  Short-term quarterly estimates tend to be more volatile, while annual estimates provide a broader view of expected performance.

ticker_obj.get_growth_estimates()

,stockTrend,indexTrend
period,,
0q,-0.3803,0.1524
+1q,0.6288,0.0703
0y,-0.3180,0.1090
+1y,0.3622,0.1487
LTG,NaN,0.1220


In [27]:
ticker_obj.get_earnings_history()

,epsActual,epsEstimate,epsDifference,surprisePercent
quarter,,,,
2024-12-31,0.73,0.76703,-0.04,-0.0483
2025-03-31,0.27,0.41468,-0.14,-0.3489
2025-06-30,0.40,0.40391,-0.00,-0.0097
2025-09-30,0.50,0.55885,-0.06,-0.1053


In [28]:
import pandas as pd
news = pd.DataFrame(ticker_obj.get_news())
news.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       10 non-null     object
 1   content  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes


In [29]:
news

,id,content
0,8ee9763c-cd44-37c1-916c-066a62206691,"{'id': '8ee9763c-cd44-37c1-916c-066a62206691',..."
1,e3d5a176-deac-351b-94ec-c966dd19ad9a,"{'id': 'e3d5a176-deac-351b-94ec-c966dd19ad9a',..."
2,fcfaad4d-beff-3edb-b346-8a9282ece837,"{'id': 'fcfaad4d-beff-3edb-b346-8a9282ece837',..."
3,3c54efeb-8d61-3f56-90ba-bb013f26d2f0,"{'id': '3c54efeb-8d61-3f56-90ba-bb013f26d2f0',..."
4,449fc3c8-8626-3dc1-8213-5369f278dcc1,"{'id': '449fc3c8-8626-3dc1-8213-5369f278dcc1',..."
5,610e508a-d27a-308d-b2fc-65ef4013f686,"{'id': '610e508a-d27a-308d-b2fc-65ef4013f686',..."
6,0b624624-c69f-3d69-a013-b73d7ba71497,"{'id': '0b624624-c69f-3d69-a013-b73d7ba71497',..."
7,db0bb69a-6f03-335f-b8ae-d40399784e9f,"{'id': 'db0bb69a-6f03-335f-b8ae-d40399784e9f',..."
8,72e0efd1-dee1-3d56-990b-5ad7060df450,"{'id': '72e0efd1-dee1-3d56-990b-5ad7060df450',..."
9,0b50e1e0-f6c6-3fdd-b3fb-54e5260a279d,"{'id': '0b50e1e0-f6c6-3fdd-b3fb-54e5260a279d',..."


In [30]:
import pandas as pd

# Create a new DataFrame from the 'content' column of the 'news' DataFrame
news_df = pd.DataFrame(news['content'].tolist())

# Display the first few rows of the new DataFrame
display(news_df.head())

,id,contentType,title,description,summary,pubDate,displayTime,isHosted,bypassModal,previewUrl,thumbnail,provider,canonicalUrl,clickThroughUrl,metadata,finance,storyline
0,8ee9763c-cd44-37c1-916c-066a62206691,VIDEO,Mag 7 stocks are 'back in favor' among retail ...,"<p>Robinhood's (<a data-i13n=""cpos:1;pos:1"" hr...","Robinhood's (HOOD) chief brokerage officer, St...",2025-12-18T22:30:00Z,,True,False,None,{'originalUrl': 'https://s.yimg.com/os/creatr-...,"{'displayName': 'Yahoo Finance Video', 'url': ...",{'url': 'https://finance.yahoo.com/video/mag-7...,{'url': 'https://finance.yahoo.com/video/mag-7...,{'editorsPick': True},"{'premiumFinance': {'isPremiumNews': False, 'i...",{'storylineItems': [{'content': {'id': '6bf9db...
1,e3d5a176-deac-351b-94ec-c966dd19ad9a,STORY,"Stock Market Today, Dec. 18: Rivian Automotive...",,"Today, Dec. 18, 2025, a bullish analyst call s...",2025-12-18T22:47:01Z,2025-12-18T22:47:01Z,True,False,None,{'originalUrl': 'https://media.zenfs.com/en/mo...,"{'displayName': 'Motley Fool', 'url': 'http://...",{'url': 'https://www.fool.com/coverage/stock-m...,{'url': 'https://finance.yahoo.com/news/stock-...,{'editorsPick': False},"{'premiumFinance': {'isPremiumNews': False, 'i...",None
2,fcfaad4d-beff-3edb-b346-8a9282ece837,STORY,Tesla’s ‘Musk Premium’ in Focus With SpaceX IP...,,Elon Musk is selling investors a future of dri...,2025-12-18T21:27:11Z,2025-12-18T21:27:11Z,True,False,None,{'originalUrl': 'https://media.zenfs.com/en/bl...,"{'displayName': 'Bloomberg', 'url': 'https://w...",{'url': 'https://finance.yahoo.com/news/tesla-...,{'url': 'https://finance.yahoo.com/news/tesla-...,{'editorsPick': False},"{'premiumFinance': {'isPremiumNews': False, 'i...",None
3,3c54efeb-8d61-3f56-90ba-bb013f26d2f0,STORY,Stock Market Today: Indexes Keep Gains; Wester...,,Stock Market Today: The Dow Jones index rose T...,2025-12-18T21:22:13Z,2025-12-18T21:22:13Z,False,False,https://finance.yahoo.com/m/3c54efeb-8d61-3f56...,{'originalUrl': 'https://media.zenfs.com/en/ib...,"{'displayName': 'Investor's Business Daily', '...",{'url': 'https://www.investors.com/market-tren...,None,{'editorsPick': False},"{'premiumFinance': {'isPremiumNews': False, 'i...",None
4,449fc3c8-8626-3dc1-8213-5369f278dcc1,STORY,Tesla Stock Rebounds After AI Selloff. Why the...,,The shares got caught up in the artificial-int...,2025-12-18T21:16:00Z,2025-12-18T21:16:00Z,False,False,https://finance.yahoo.com/m/449fc3c8-8626-3dc1...,{'originalUrl': 'https://media.zenfs.com/en/Ba...,"{'displayName': 'Barrons.com', 'url': 'http://...",{'url': 'https://www.barrons.com/articles/tesl...,None,{'editorsPick': False},"{'premiumFinance': {'isPremiumNews': False, 'i...",None


In [31]:
# summary of news articles for the ticker (Yahoo Finance)
from pprint import pprint 
for e in news_df.summary:
    pprint(e)
    print('-----')

("Robinhood's (HOOD) chief brokerage officer, Steve Quirk, joins Market "
 'Domination host Josh Lipton to talk about trends among retail investors '
 'before 2025 draws to a close. Watch the video above to hear more about '
 'retail investor behavior on the platform. To watch more expert insights and '
 'analysis on the latest market action, check out more Market Domination.')
-----
('Today, Dec. 18, 2025, a bullish analyst call spotlights Rivian’s R2 roadmap, '
 'autonomy push, and evolving EV strategy.')
-----
('Elon Musk is selling investors a future of driverless cars, robot assistants '
 'and life on Mars.  The problem for investors is there’s only one way to buy '
 'into his vision: Tesla Inc.’s stock.  The rally is a testament to Wall '
 'Street’s faith in Musk’s artificial intelligence ambitions, particularly '
 'since Tesla’s core auto business is struggling.')
-----
('Stock Market Today: The Dow Jones index rose Thursday on a surprise CPI '
 'inflation report. Nvidia stock a

In [32]:
# Fast info about the stock
ticker_obj.get_fast_info().items()

[('currency', 'USD'),
 ('dayHigh', 490.8599853515625),
 ('dayLow', 473.1199951171875),
 ('exchange', 'NMS'),
 ('fiftyDayAverage', 437.4781643614477),
 ('lastPrice', 483.3699951171875),
 ('lastVolume', 94595726),
 ('marketCap', 1607601194513.4385),
 ('open', 477.8999938964844),
 ('previousClose', 467.4),
 ('quoteType', 'EQUITY'),
 ('regularMarketPreviousClose', 467.260009765625),
 ('shares', 3325819167),
 ('tenDayAverageVolume', 70774490),
 ('threeMonthAverageVolume', 84250503),
 ('timezone', 'America/New_York'),
 ('twoHundredDayAverage', 348.44206053767374),
 ('yearChange', 0.07127953637519303),
 ('yearHigh', 495.2799987792969),
 ('yearLow', 214.25)]

In [35]:
import json

In [36]:
# a lot of info - potentially useful fields:
# website, industry, sector, fullTimeEmployees, companyOfficers, previousClose, open, dayLow, dayHigh, 
# beta, forwardPE, trailingPE, volume, averageVolume, averageVolume10days, marketCap, fiftyTwoWeekLow, 
# fiftyTwoWeekHigh, allTimeHigh, allTimeLow, priceToSalesTrailing12Months, fiftyDayAverage, twoHundredDayAverage
# profitMargins, sharesPercentSharesOut, heldPercentInsiders, heldPercentInstitutions, shortRatio, bookValue, priceToBook
# earningsQuarterlyGrowth, trailingEps, forwardEps, 52WeekChange, currentPrice,
#   targetHighPrice, targetLowPrice, targetMeanPrice, targetMedianPrice, recommendationMean,
# recommendationKey, numberOfAnalystOpinions, totalCashPerShare,totalCashPerShare, returnOnAssets, returnOnEquity,
# earningsGrowth, revenueGrowth, grossMargins, ebitdaMargins, operatingMargins, region, fullExchangeName, regularMarketDayRange, fiftyTwoWeekRange,
# fiftyTwoWeekHighChange, cryptoTradeable, displayName, trailingPegRatio
info = ticker_obj.get_info()
print(json.dumps(info, indent=2, default=str))

{
  "address1": "1 Tesla Road",
  "city": "Austin",
  "state": "TX",
  "zip": "78725",
  "country": "United States",
  "phone": "512 516 8177",
  "website": "https://www.tesla.com",
  "industry": "Auto Manufacturers",
  "industryKey": "auto-manufacturers",
  "industryDisp": "Auto Manufacturers",
  "sector": "Consumer Cyclical",
  "sectorKey": "consumer-cyclical",
  "sectorDisp": "Consumer Cyclical",
  "longBusinessSummary": "Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, body shop and parts, supercharging, retail merchandise, and vehicle insurance services. This segment also provides sedans and sport utility vehicles through direc

In [ ]:
# more history metadata - not sure we need this now
import json

metadata = ticker_obj.get_history_metadata()
print(json.dumps(metadata, indent=2, default=str))

{
  "currency": "USD",
  "symbol": "TSLA",
  "exchangeName": "NMS",
  "fullExchangeName": "NasdaqGS",
  "instrumentType": "EQUITY",
  "firstTradeDate": 1277818200,
  "regularMarketTime": 1766005201,
  "hasPrePostMarketData": true,
  "gmtoffset": -18000,
  "timezone": "EST",
  "exchangeTimezoneName": "America/New_York",
  "regularMarketPrice": 467.26,
  "fiftyTwoWeekHigh": 495.28,
  "fiftyTwoWeekLow": 214.25,
  "regularMarketDayHigh": 495.24,
  "regularMarketDayLow": 466.2,
  "regularMarketVolume": 104916333,
  "longName": "Tesla, Inc.",
  "shortName": "Tesla, Inc.",
  "chartPreviousClose": 446.89,
  "previousClose": 489.88,
  "scale": 3,
  "priceHint": 2,
  "currentTradingPeriod": {
    "pre": {
      "timezone": "EST",
      "start": 1766048400,
      "end": 1766068200,
      "gmtoffset": -18000
    },
    "regular": {
      "timezone": "EST",
      "start": 1766068200,
      "end": 1766091600,
      "gmtoffset": -18000
    },
    "post": {
      "timezone": "EST",
      "start": 1766